# Mise en place d'une réplication logique

Dans ce notebook, nous allons mettre en place une réplication logique en commençant par l'uni-directionnelle (d'une instance vers une autre, sans retour).

Ensuite, nous essayerons (bêtement) de créer une réplication dans l'autre sens et nous verrons rapidement que cela ne peut pas fonctionner en l'état, sans une option rajoutée dans PostgreSQL 16.0.

## Installation et configuration

Nous allons déjà débuter par configurer notre notebook, et créer les deux objets matérialisant nos deux instances PostgreSQL.

In [1]:
%load_ext sql
import sqlalchemy

In [2]:
engine_pg_na = sqlalchemy.create_engine('postgresql://postgres:postgres@pg_na:5432/postgres')
engine_pg_western_europe = sqlalchemy.create_engine('postgresql://postgres:postgres@pg_western_europe:5432/postgres')


## Données initiales

Une fois que cela est fait, nous allons partir en Europe de l'Ouest, créer notre table et y insérer quelques tuples.

In [3]:
%sql engine_pg_western_europe

In [4]:
%%sql DROP TABLE IF EXISTS ticket; CREATE TABLE ticket(
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    total NUMERIC(5,2)
)


Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

++
||
++
++

In [5]:
%sql INSERT INTO ticket(total) VALUES (50.23), (22.3);

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

2 rows affected.

++
||
++
++

In [6]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

2 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30


## Création de la publication

Notre table est prête et contient déjà quelques données. On va maintenant créer une publication pour que notre instance nord-américaine puisse se synchroniser avec les changements locaux. 

In [7]:
%sql CREATE PUBLICATION pub_ticket_western_europe FOR TABLE ticket;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

++
||
++
++

## Création de la souscription

Maintenant, nous pouvons nous connecter à l'instance nord-américaine pour mettre en place la souscription.
On va commencer par créer une table identique à celle déjà créée plus haut.
    
> À noter que dans la réplication logique, les opération de DDL (Data Definition Language) ne sont pas répliquées puisque les opérations sont streamées au niveau des tuples uniquement.

In [8]:
%sql engine_pg_na

In [9]:
%%sql DROP TABLE IF EXISTS ticket; CREATE TABLE ticket(
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    total NUMERIC(5,2)
)


Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

++
||
++
++

In [10]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

id,total


On va pouvoir souscrire à la publication réalisée plus haut sur l'autre instance.

In [11]:
%%sql CREATE SUBSCRIPTION sub_ticket_western_europe
    CONNECTION 'host=pg_western_europe port=5432 user=postgres'
    PUBLICATION pub_ticket_western_europe;

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

++
||
++
++

Lorsqu'on crée une souscription sur un noeud, le processus commence par prendre un instantané de la table sur le noeud publicateur. Pour vérifier que cela fonctionne correctement, regardons le contenu de notre table :

In [12]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

2 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30


Nous avons bien les données insérées plus haut dans la base `pg_western_europe`. Maintenant, que se passe-t-il si on essaie de faire un insert ?

In [13]:
%sql INSERT INTO ticket (total) VALUES (15.32) ON CONFLICT DO NOTHING; SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

1 rows affected.

3 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30
23b49661-8c5b-41ab-8132-a1fbf11d8439,15.32


Nous avons bien notre nouveau ticket. Par contre, il est logique qu'il ne soit pas répliqué de l'autre côté de l'Atlantique puisque nous n'avons pas encore réalisé la route "retour". C'est ce que nous allons faire à présent.

In [14]:
%sql CREATE PUBLICATION pub_ticket_na FOR TABLE ticket;

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

++
||
++
++

In [15]:
%sql engine_pg_western_europe

In [16]:
%%sql CREATE SUBSCRIPTION sub_ticket_na
    CONNECTION 'host=pg_na port=5432 user=postgres'
    PUBLICATION pub_ticket_na;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

++
||
++
++

In [17]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

2 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30


Misère, notre ligne rajoutée plus haut ne remonte pas. En analysant les logs de notre instance pg-western-europe, on voit ce genre de messages (peut varier car l'UUID est autogénéré)

```
pg-western-europe-1 |2025-02-05 11:04:03.441 UTC [84] LOG:  logical replication table synchronization worker for subscription "sub_ticket_na", table "ticket" has started
pg-western-europe-1 |2025-02-05 11:04:03.465 UTC [84] ERROR:  duplicate key value violates unique constraint "ticket_pkey"
pg-western-europe-1 |2025-02-05 11:04:03.465 UTC [84] DETAIL:  Key (id)=(c7eb8a92-35ea-4be5-9c51-1691f645381a) already exists.
pg-western-europe-1 |2025-02-05 11:04:03.465 UTC [84] CONTEXT:  COPY ticket, line 1
```

On arrive au problème inhérent à la réplication bi-directionnelle : la boucle de réplication. On essaie là de répliquer une donnée déjà présente sur le souscriveur, on reçoit donc une erreur liée à l'unicité des clés primaires. On va donc utiliser l'option `copy_data` pour désactiver la copie initiale, ce qui pourrait nécessiter en conditions réelles de faire une synchronisation par un autre biais.



In [18]:
%sql DROP SUBSCRIPTION sub_ticket_na

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

++
||
++
++

In [19]:
%%sql CREATE SUBSCRIPTION sub_ticket_na
    CONNECTION 'host=pg_na port=5432 user=postgres'
    PUBLICATION pub_ticket_na
    WITH (copy_data=false, ORIGIN=none);

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

++
||
++
++

In [20]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

2 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30


In [21]:
%sql engine_pg_na

In [22]:
%sql INSERT INTO ticket(total) VALUES(14.34);

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

1 rows affected.

++
||
++
++

In [23]:
%sql engine_pg_western_europe

In [24]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

3 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30
15d85d16-aba1-481b-b5d4-4b8303a8fc1b,14.34


Superbe, maintenant on obtient bien dans la base `pg_western_europe` les lignes insérées dans la base `pg_na`. Par contre, on arrive sur un petit souci dans celle-ci...

```
pg-na-1 |2025-02-05 12:44:34.052 UTC [91] LOG:  logical replication apply worker for subscription "sub_ticket_western_europe" has started
pg-na-1 |2025-02-05 12:44:34.060 UTC [91] ERROR:  duplicate key value violates unique constraint "ticket_pkey"
pg-na-1 |2025-02-05 12:44:34.060 UTC [91] DETAIL:  Key (id)=(15d85d16-aba1-481b-b5d4-4b8303a8fc1b) already exists.
pg-na-1 |2025-02-05 12:44:34.060 UTC [91] CONTEXT:  processing remote data for replication origin "pg_16394" during message type "INSERT" for replication target relation "public.ticket" in transaction 762, finished at 0/155A788
pg-na-1 |2025-02-05 12:44:34.062 UTC [1] LOG:  background worker "logical replication apply worker" (PID 91) exited with exit code 1

```

La ligne répliquée sur la base `pg_western_europe` revient sur la base `pg_na` car nous avions déjà un canal de réplication d'ouvert.

C'est là que PostgreSQL 16.x introduit une amélioration, avec l'ajout de l'option ORIGIN dans les souscriptions ! On va donc dropper la souscription existante pour en recréer une avec la bonne configuration.


In [25]:
%sql engine_pg_na

In [26]:
%sql DROP SUBSCRIPTION sub_ticket_western_europe

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

++
||
++
++

In [27]:
%%sql CREATE SUBSCRIPTION sub_ticket_western_europe
    CONNECTION 'host=pg_western_europe port=5432 user=postgres'
    PUBLICATION pub_ticket_western_europe
    WITH (copy_data=false, ORIGIN=none);

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

++
||
++
++

Maintenant, on va réessayer d'insérer une ligne côté NA et s'assurer que nous n'avons pas le problème de retour.

In [28]:
%sql INSERT INTO ticket(total) VALUES(17.28);

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

1 rows affected.

++
||
++
++

In [29]:
%sql engine_pg_western_europe

In [30]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

4 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30
15d85d16-aba1-481b-b5d4-4b8303a8fc1b,14.34
91f21a82-41ac-4d6b-82d5-9859eea13f2d,17.28


In [31]:
%sql INSERT INTO ticket(total) VALUES(20.31);

Running query in 'postgresql://postgres:***@pg_western_europe:5432/postgres'

1 rows affected.

++
||
++
++

In [32]:
%sql engine_pg_na

In [33]:
%sql SELECT * FROM ticket;

Running query in 'postgresql://postgres:***@pg_na:5432/postgres'

6 rows affected.

id,total
cce89557-b5ce-4897-a648-2166d2665451,50.23
50344fcd-17e4-485e-a262-18427e3d7ced,22.30
23b49661-8c5b-41ab-8132-a1fbf11d8439,15.32
15d85d16-aba1-481b-b5d4-4b8303a8fc1b,14.34
91f21a82-41ac-4d6b-82d5-9859eea13f2d,17.28
d6a46101-724f-4bef-ab8e-9c641d9e9ff2,20.31


La ligne avec `total=17.28` créée sur `pg_na` a bien été répliquée sur `pg_western_europe`, et on a même pu valider que le process n'était pas bloqué en créant une ligne (`total=20.31`) sur `pg_western_europe` qui s'est bien retrouvée de l'autre côté.